[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[documentation for python-ecdsa](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [91]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

message = b"Don't touch that XRP"
signature = sk.sign(message)

In [92]:
vk.verify(signature, message)

True

In [93]:
vk.verify(signature, b"Nice XRP")

BadSignatureError: 

# Using Unique Serial Number as Our initial Message

* We use uuid's because they're easy to produce and very unlikely to collide

# Using signatures of signatures as subsequent messages

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. But where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spent the coin you just need to serialize the coin, and sign it using the private key corresponding the the `public_key` listed in the last transfer.

In [25]:
import uuid

def generate_id():
    return uuid.uuid4()

class Transfer:
    
    def __init__(self, message, signature, public_key):
        self.message = message
        self.signature = signature
        self.public_key = public_key

class ECDSACoin:
    
    def __init__(self, id, transfers):
        self.id = id 
        self.transfers = transfers  # Instances of Transfer ^^

In [26]:
from ecdsa import SigningKey, SECP256k1

# These are comically small numbers ...
# Doing this to point out that they're just numbers
bank_secret = 1
bob_secret = 2
alice_secret = 3

bank_sk = SigningKey.from_secret_exponent(bank_secret, curve=SECP256k1)
bob_sk = SigningKey.from_secret_exponent(bob_secret, curve=SECP256k1)
alice_sk = SigningKey.from_secret_exponent(alice_secret, curve=SECP256k1)

bank_vk = bank_sk.get_verifying_key()
bob_vk = bob_sk.get_verifying_key()
alice_vk = alice_sk.get_verifying_key()

In [27]:
import pickle

# coin id -> public key issued to
circulation = {}

# Only the bank can do this
def issue(public_key):
    # Create a message specifying which coin we're issuing (id) 
    # and to whom we're issuing it
    id = generate_id()
    circulation[id] = public_key
    data = {
        "id": id,
        "public_key": public_key,
    }
    message = pickle.dumps(data)
    
    # create the first transfer, which includes a special "minting" signature
    signature = bank_sk.sign(message)
    transfer = Transfer(
        message=message,
        signature=signature,
        public_key=public_key,
    )
    
    # Create and return the coin with just the issuing transfer
    coin = ECDSACoin(id=id, transfers=[transfer])
    return coin
    


# Validating the First / Minting Transfer

In [28]:
# Introduce the bank, which maintains a 

In [38]:
def validate_issuing_transfer(coin):
    transfer = coin.transfers[0]
    data = {
        "id": coin.id,
        "public_key": transfer.public_key,
    }
    message = pickle.dumps(data)
    assert bank_vk.verify(transfer.signature, message)

In [39]:
# give alice a ECDSACoin
alice_coin = issue(alice_vk)

validate_issuing_transfer(alice_coin)

In [ ]:
# Example where the signature is done using wrong private key

bad_coin_id = generate_id()
bad_message = pickle.dumps({
    "id": id,
    "public_key": alice_vk,
})
bad_transfer = Transfer(
    message=bad_message,
    signature=bob_sk.sign(bad_message),  # WRONG SIGNING KEY
    public_key=alice_vk,
)
bad_coin = ECDSACoin(
    id=bad_coin_id,
    transfers=[bad_transfer],
)

In [75]:
from ecdsa import BadSignatureError

try:
    # Should raise a BadSignatureError
    validate_issuing_transfer(bad_coin)
except BadSignatureError:
    print("It caught the bad signature")

It caught the bad signature


# Validating subsequent Transfers

In [78]:
def validate_transfers(coin):
    validate_issuing_transfer(coin)
    previous_transfer = coin.transfers[0]
    for transfer in coin.transfers[1:]:
        # Check previous owner signed this transfer using their private key
        assert previous_transfer.public_key.verify(
            transfer.signature,
            previous_transfer.signature,
        )
        previous_transfer = transfer

In [70]:
alice_coin = issue(alice_vk)

alice_to_bob = Transfer(
    message=None,
    signature=alice_sk.sign(alice_coin.transfers[0].signature),
    public_key=bob_vk,
)

alice_coin.transfers.append(alice_to_bob)

b'\x8c\x15^\x1b\x98\x96\xf3g\x8a\xf5R\x1aK\x83\x0e\xf4s(}\x8c&9\x98!&R\x1b\x9b8\x1b\xe0[ez\xd5L\x14\xa6\x82p\xdb\x15~.\xfb\xdf{N)\xcb\xc8\xb7\x9aS/\xa4\xac\x87\xcb\x8e\xb6d\\\xa9'


In [71]:
# No errors means the signature was good
validate_transfers(alice_coin)

b'\x8c\x15^\x1b\x98\x96\xf3g\x8a\xf5R\x1aK\x83\x0e\xf4s(}\x8c&9\x98!&R\x1b\x9b8\x1b\xe0[ez\xd5L\x14\xa6\x82p\xdb\x15~.\xfb\xdf{N)\xcb\xc8\xb7\x9aS/\xa4\xac\x87\xcb\x8e\xb6d\\\xa9'


In [72]:
alice_coin = issue(alice_vk)

alice_to_bob_forged = Transfer(
    message=None,
    signature=bob_sk.sign(alice_coin.transfers[0].signature),
    public_key=bob_vk,
)

alice_coin.transfers.append(alice_to_bob_forged)

In [79]:
try:
    # Should raise a BadSignatureError
    validate_transfers(alice_coin)
except BadSignatureError:
    print("It caught the bad signature")

It caught the bad signature


# Serialization

In [80]:
# The code from last time still works (utils.py)

from utils import to_disk, from_disk

In [81]:
import os

filename = "alices.ecdsacoin"
 
print("Alice's coinfile exists on disk?", os.path.isfile(filename))

Alice's coinfile exists on disk? False


In [83]:
alice_coin = issue(alice_vk)

to_disk(alice_coin, filename)

In [84]:
print("Alice's coinfile exists on disk?", os.path.isfile(filename))

Alice's coinfile exists on disk? True


# The Finished Produce

(link to library)